# メソッド分離
やっていること
1. データセットのディレクトリから全javaファイルのパスを取得。
2. 取得したものをlizardにかけてメソッドの行数を取得
3. ファイルを読み込んでメソッド行数でスプリット
4. 結果をDataframeに格納
データ構造
```typescript
dataframeType = {
    style: "training" | "validation" | "test";
    methods: string; // '    public VerifierException(String message)\n    {\n        super(message);\n    }\n'
}
```

In [1]:
import pandas as pd
import numpy as np
import lizard
import glob
from pprint import pprint
from typing import List

In [2]:
JAVA_SMALL_RAW_DATA = "../../data/java-small_raw" # Data path of java-small_raw
STYLES = ["training","validation","test"] 
SAVE_DATA_PATH = "../../data/1_devided_methods"

In [3]:
def get_methods_files(filepath :str) -> List[str]:
    liz = lizard.analyze_file(filepath,)
    source_code = []
    with open(filepath,encoding="utf8", errors='ignore') as f:
        source_code = f.readlines()
    methods = []
    methods.extend( ["".join(source_code[liz_func.start_line-1: liz_func.end_line]) for liz_func in liz.function_list])
    return methods

In [4]:

df_methods = None
for style in STYLES:
    filepaths = glob.glob(f"{JAVA_SMALL_RAW_DATA}/{style}/**/*")
    print(style)
    for i,filepath in enumerate(filepaths):
        if i%1000 == 0:print(i) 
        methods = get_methods_files(filepath)
        temp_df = pd.DataFrame({"style": [style]*len(methods),"method_code":methods})
        if df_methods is None:
            df_methods = temp_df
        else:
            df_methods = pd.concat([df_methods, temp_df])
df_methods=df_methods.reset_index(drop=True)
df_methods.to_pickle(f"{SAVE_DATA_PATH}/1_devided_methods.pickle")


training
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
validation
0
1000
test
0
1000
2000
3000
4000
5000


In [ ]:
df_methods